## ●Notebookの内容

変数選択後データ(コード4の「回帰分析による変数選択」で作成したもの)の読み込み、前処理(ランク正規化)、形状変換

# 1. パッケージのインポート・乱数固定

In [1]:
import pandas as pd #pandasパッケージをインポート
import numpy as np #numpyパッケージをインポート
import torch #ライブラリ「PyTorch」のtorchパッケージをインポート
import torch.nn as nn #「ニューラルネットワーク」モジュールの別名定義
import torch.nn.functional as F #「ニューラルネットワーク・活性化関数」モジュールの別名定義
import collections
import os
import pickle
import optuna
import torch.optim as optim

#乱数の固定
torch.manual_seed(123)

# 2. 整理後データの読み込み

In [2]:
pickle_in = open("./organized_data3.pickle","rb") 
data_4 = pickle.load(pickle_in)
pickle_in.close()
display(data_4)

,銘柄名,期日,銘柄コード,調整済始値,調整済高値,調整済安値,株価(終値)移動平均25日乖離率,株価(終値)移動平均75日乖離率,株価(終値)移動平均200日乖離率,PER(普通株式数･日経予想･連結優先･含負),...,総還元性向,総資産連単倍率,売上高連単倍率,営業利益連単倍率,経常利益連単倍率,当期利益連単倍率,期末EV/EBITDA,時価総額(発行済株式数ﾍﾞｰｽ),計算用現値（調整済、市場引継）,対TOPIX β値(60ヵ月) 2乗
0,極洋,1994/12,1301.0,3780.0,4190.0,3710.0,2.9185,-1.3674,-7.4138,60.719152,...,34.644268,1.057022,1.049759,1.141331,1.049765,0.946636,35.878976,4.553936e+10,4020.0,1.640167
1,極洋,1995/01,1301.0,4070.0,4130.0,3580.0,-2.8197,-3.9003,-10.0966,58.302469,...,34.801282,1.057022,1.049759,1.141445,1.049182,0.946636,35.878976,4.372685e+10,3860.0,1.621369
2,極洋,1995/02,1301.0,3880.0,4000.0,3460.0,-8.3524,-10.2346,-16.9429,53.167019,...,34.801282,1.057022,1.049759,1.141100,1.047150,0.946636,35.878976,3.987526e+10,3520.0,1.728384
3,極洋,1995/03,1301.0,3520.0,3800.0,3350.0,-3.1339,-10.3753,-17.6057,51.354507,...,34.424683,1.057022,1.049759,1.141879,1.049182,0.946636,35.878976,3.851588e+10,3400.0,1.583596
4,極洋,1995/04,1301.0,3300.0,3700.0,3150.0,1.7775,-3.0125,-11.4933,53.620147,...,34.424683,1.057022,1.049759,1.141879,1.049406,0.946636,35.878976,4.021511e+10,3550.0,1.579018
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
370820,ベルーナ,2021/08,9997.0,883.0,900.0,773.0,-2.3335,-14.7397,-21.8114,6.358036,...,14.452700,1.123884,1.598314,1.584172,1.541385,1.483134,8.300202,7.993457e+10,822.0,2.385110
370821,ベルーナ,2021/09,9997.0,822.0,892.0,819.0,0.2582,-4.7101,-18.3318,6.605550,...,14.452700,1.123884,1.598314,1.584172,1.541385,1.483134,8.300202,8.304638e+10,854.0,2.379800
370822,ベルーナ,2021/10,9997.0,848.0,851.0,792.0,-3.3851,-6.3116,-22.2559,6.172400,...,14.452700,1.123884,1.598314,1.584172,1.541385,1.483134,8.300202,7.760071e+10,798.0,2.348832
370823,ベルーナ,2021/11,9997.0,793.0,795.0,665.0,-10.6664,-17.3400,-32.4331,5.159130,...,14.452700,1.123884,1.598314,1.584172,1.541385,1.483134,8.300202,6.486175e+10,667.0,2.758423


In [3]:
day_num_4 = data_4.期日.nunique(dropna = True) #期日の数(月数)
com_num_4 = data_4.shape[0]/day_num_4 #上場廃止や途中上場がない使用可能な企業数
#com_num_4 = data_4.銘柄コード.nunique(dropna = True) #上場廃止や途中上場がない使用可能な企業数
print('使用可能な企業数は', com_num_4,'、', '期日数は', day_num_4 ,'である。')

使用可能な企業数は 1141.0 、 期日数は 325 である。


# 3. 前処理(ランク正規化(-1,1))と書き出し

## 3.1 ランク正規化(0,1)の実行

In [4]:
data_4_1 = data_4.sort_values(by=['期日'])
data_4_1 = data_4_1.reset_index(drop=True) #indexの振り直し(元のindexを新しい列にしてそれを削除
data_4_6 = pd.DataFrame()
for e in range(0,day_num_4):
    data_4_2 = data_4_1.loc[0+(com_num_4*e):com_num_4-1+(com_num_4*e),'調整済始値':]
    data_4_3 = data_4_1.loc[0+(com_num_4*e):com_num_4-1+(com_num_4*e),:'銘柄コード']
    data_4_4 = data_4_2.rank(pct = True)
    data_4_5 = pd.concat([data_4_3,data_4_4],axis=1)
    data_4_6 = pd.concat([data_4_6,data_4_5],axis=0)
display(data_4_6)
data_4_7_0 = data_4_6.sort_values(by=['銘柄コード', '期日']) #ランク正規化したものを期日と銘柄コードをキーに並べ替え
data_4_7_0 = data_4_7_0.reset_index(drop=True) #indexの振り直し(元のindexを新しい列にしてそれを削除
display(data_4_7_0) #期日と銘柄コードをキーに並べ替えしたものを表示

,銘柄名,期日,銘柄コード,調整済始値,調整済高値,調整済安値,株価(終値)移動平均25日乖離率,株価(終値)移動平均75日乖離率,株価(終値)移動平均200日乖離率,PER(普通株式数･日経予想･連結優先･含負),...,総還元性向,総資産連単倍率,売上高連単倍率,営業利益連単倍率,経常利益連単倍率,当期利益連単倍率,期末EV/EBITDA,時価総額(発行済株式数ﾍﾞｰｽ),計算用現値（調整済、市場引継）,対TOPIX β値(60ヵ月) 2乗
0,極洋,1994/12,1301.0,0.773883,0.784400,0.778703,0.567046,0.376862,0.325153,0.696757,...,0.500438,0.206836,0.156880,0.500438,0.500438,0.230500,0.955302,0.411919,0.784400,0.781770
1,日精樹脂,1994/12,6293.0,0.453550,0.463190,0.462314,0.051709,0.178791,0.799299,0.799299,...,0.650307,0.105171,0.199825,0.868536,0.915863,0.974584,0.343558,0.223488,0.449606,0.056091
2,ウシオ電,1994/12,6925.0,0.333918,0.353637,0.334356,0.956179,0.974584,0.965819,0.552147,...,0.830850,0.669588,0.963190,0.290096,0.174408,0.156880,0.664330,0.761613,0.369851,0.863278
3,クニミネ,1994/12,5388.0,0.469325,0.481595,0.469325,0.017528,0.536372,0.870289,0.639790,...,0.665206,0.500438,0.500438,0.500438,0.500438,0.500438,0.338300,0.142857,0.444785,0.500438
4,第一実,1994/12,8059.0,0.723488,0.720859,0.728309,0.325153,0.239264,0.802805,0.815074,...,0.915863,0.035933,0.036810,0.078002,0.133216,0.163015,0.240140,0.319895,0.720421,0.829097
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
370820,ＣＫＤ,2021/12,6407.0,0.662577,0.671341,0.673094,0.567046,0.705521,0.635408,0.588957,...,0.339176,0.350570,0.278703,0.646801,0.631025,0.727432,0.766871,0.687993,0.667835,0.827344
370821,太陽誘電,2021/12,6976.0,0.943909,0.943909,0.943909,0.661700,0.834356,0.901840,0.692375,...,0.062226,0.680982,0.170026,0.804557,0.840491,0.875548,0.713409,0.893953,0.949167,0.618755
370822,栃木銀,2021/12,8550.0,0.012270,0.021911,0.014023,0.995618,0.988606,0.963190,0.209465,...,0.196319,0.047327,0.185802,0.500438,0.609115,0.695004,0.500438,0.266433,0.022787,0.801052
370823,帝人,2021/12,3401.0,0.410167,0.408852,0.415425,0.723926,0.310254,0.120947,0.197195,...,0.500438,0.902717,0.895706,0.500438,0.923751,0.500438,0.352323,0.784400,0.426819,0.427695


,銘柄名,期日,銘柄コード,調整済始値,調整済高値,調整済安値,株価(終値)移動平均25日乖離率,株価(終値)移動平均75日乖離率,株価(終値)移動平均200日乖離率,PER(普通株式数･日経予想･連結優先･含負),...,総還元性向,総資産連単倍率,売上高連単倍率,営業利益連単倍率,経常利益連単倍率,当期利益連単倍率,期末EV/EBITDA,時価総額(発行済株式数ﾍﾞｰｽ),計算用現値（調整済、市場引継）,対TOPIX β値(60ヵ月) 2乗
0,極洋,1994/12,1301.0,0.773883,0.784400,0.778703,0.567046,0.376862,0.325153,0.696757,...,0.500438,0.206836,0.156880,0.500438,0.500438,0.230500,0.955302,0.411919,0.784400,0.781770
1,極洋,1995/01,1301.0,0.785276,0.780894,0.788782,0.626643,0.632778,0.537248,0.717791,...,0.500438,0.206836,0.156880,0.500438,0.500438,0.230500,0.955302,0.417178,0.789220,0.765995
2,極洋,1995/02,1301.0,0.787905,0.788782,0.791411,0.223488,0.462752,0.433830,0.705521,...,0.500438,0.207713,0.156004,0.500438,0.500438,0.228747,0.955302,0.416301,0.788782,0.797546
3,極洋,1995/03,1301.0,0.789658,0.800613,0.809816,0.382997,0.457493,0.544259,0.725679,...,0.500438,0.208589,0.153374,0.500438,0.500438,0.226994,0.952673,0.424189,0.790973,0.725679
4,極洋,1995/04,1301.0,0.793164,0.799737,0.792726,0.611744,0.670465,0.706398,0.747590,...,0.500438,0.208589,0.153374,0.500438,0.500438,0.226994,0.953550,0.446976,0.801928,0.729185
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
370820,ベルーナ,2021/08,9997.0,0.222612,0.213409,0.190184,0.145486,0.012270,0.009641,0.086766,...,0.034181,0.304119,0.589833,0.639790,0.693252,0.784400,0.651183,0.553900,0.196319,0.892200
370821,ベルーナ,2021/09,9997.0,0.195004,0.195004,0.198072,0.635408,0.105171,0.014899,0.090272,...,0.034181,0.304119,0.590710,0.639790,0.693252,0.785276,0.651183,0.560035,0.203330,0.891323
370822,ベルーナ,2021/10,9997.0,0.203330,0.188431,0.198948,0.172656,0.127958,0.014899,0.092025,...,0.034181,0.304119,0.590710,0.640666,0.693252,0.786152,0.652060,0.548642,0.186678,0.894829
370823,ベルーナ,2021/11,9997.0,0.180543,0.170026,0.161262,0.125329,0.068361,0.006135,0.093777,...,0.034181,0.304119,0.590710,0.638913,0.695004,0.787029,0.652936,0.530237,0.158633,0.933392


## 3.2 ランク正規化(0,1)の範囲を(-1,1)へ変更

In [5]:
data_4_7_3 = pd.DataFrame()
for g in range(data_4_7_0.iloc[:,3:].shape[1]):
    data_4_7_1 = data_4_7_0.iloc[:,3+g]
    data_4_7_2 = ((data_4_7_1-data_4_7_1.min())/(data_4_7_1.max()-data_4_7_1.min()))*(1-(-1))+(-1)
    data_4_7_3 = pd.concat([data_4_7_3, data_4_7_2], axis = 1)
data_4_7 = pd.concat([data_4_7_0.iloc[:,:3], data_4_7_3], axis=1)
display(data_4_7)

,銘柄名,期日,銘柄コード,調整済始値,調整済高値,調整済安値,株価(終値)移動平均25日乖離率,株価(終値)移動平均75日乖離率,株価(終値)移動平均200日乖離率,PER(普通株式数･日経予想･連結優先･含負),...,総還元性向,総資産連単倍率,売上高連単倍率,営業利益連単倍率,経常利益連単倍率,当期利益連単倍率,期末EV/EBITDA,時価総額(発行済株式数ﾍﾞｰｽ),計算用現値（調整済、市場引継）,対TOPIX β値(60ヵ月) 2乗
0,極洋,1994/12,1301.0,0.547368,0.568421,0.557018,0.133333,-0.247368,-0.350877,0.392982,...,0.000000,-0.587719,-0.687719,0.000000,0.000000,-0.540351,0.910526,-0.177193,0.568421,0.563158
1,極洋,1995/01,1301.0,0.570175,0.561404,0.577193,0.252632,0.264912,0.073684,0.435088,...,0.000000,-0.587719,-0.687719,0.000000,0.000000,-0.540351,0.910526,-0.166667,0.578070,0.531579
2,極洋,1995/02,1301.0,0.575439,0.577193,0.582456,-0.554386,-0.075439,-0.133333,0.410526,...,0.000000,-0.585965,-0.689474,0.000000,0.000000,-0.543860,0.910526,-0.168421,0.577193,0.594737
3,極洋,1995/03,1301.0,0.578947,0.600877,0.619298,-0.235088,-0.085965,0.087719,0.450877,...,0.000000,-0.584211,-0.694737,0.000000,0.000000,-0.547368,0.905263,-0.152632,0.581579,0.450877
4,極洋,1995/04,1301.0,0.585965,0.599123,0.585088,0.222807,0.340351,0.412281,0.494737,...,0.000000,-0.584211,-0.694737,0.000000,0.000000,-0.547368,0.907018,-0.107018,0.603509,0.457895
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
370820,ベルーナ,2021/08,9997.0,-0.556140,-0.574561,-0.621053,-0.710526,-0.977193,-0.982456,-0.828070,...,-0.933333,-0.392982,0.178947,0.278947,0.385965,0.568421,0.301754,0.107018,-0.608772,0.784211
370821,ベルーナ,2021/09,9997.0,-0.611404,-0.611404,-0.605263,0.270175,-0.791228,-0.971930,-0.821053,...,-0.933333,-0.392982,0.180702,0.278947,0.385965,0.570175,0.301754,0.119298,-0.594737,0.782456
370822,ベルーナ,2021/10,9997.0,-0.594737,-0.624561,-0.603509,-0.656140,-0.745614,-0.971930,-0.817544,...,-0.933333,-0.392982,0.180702,0.280702,0.385965,0.571930,0.303509,0.096491,-0.628070,0.789474
370823,ベルーナ,2021/11,9997.0,-0.640351,-0.661404,-0.678947,-0.750877,-0.864912,-0.989474,-0.814035,...,-0.933333,-0.392982,0.180702,0.277193,0.389474,0.573684,0.305263,0.059649,-0.684211,0.866667


## 3.3 ランク正規化後データの書き出し

In [6]:
data_4_7.to_pickle('./ranked_data-2.pickle') #ファイルの書き出し, 配当込み収益率もランク正規化したものであることに注意

In [7]:
print(os.path.getsize('./ranked_data-2.pickle')) #ファイルの大きさを表示

263548894


# 4. 分析対象となる形状に変換

## 4.1 右側ファクターネットワークの入力データ

In [8]:
#returns_data_1 = data_4_7['配当込み収益率(月次)'] #収益率データのみ抽出, ランク正規化済
returns_data_1 = data_4['配当込み収益率(月次)'] #収益率データのみ抽出, ランク正規化未
returns_data_2 = np.split(returns_data_1, com_num_4, 0)
returns_data_3 = pd.DataFrame(returns_data_2[0])
for num in range(int(com_num_4)):
    returns_data_3 = np.concatenate([returns_data_3,pd.DataFrame(returns_data_2[num])], 1)
returns_data_4 = pd.DataFrame(returns_data_3)
returns_data_4 = returns_data_4.drop(returns_data_4.columns[[0]], axis=1) 
x = returns_data_4.to_numpy()
x.shape

(325, 1141)

## 4.2 左側ベータネットワークの入力データ

In [9]:
data_4_8 = data_4_7.drop(data_4_7.columns[[0, 1, 2]], axis=1) #余分な列を削除
data_4_8 = data_4_8.drop('配当込み収益率(月次)', axis=1) #配当込み収益率データを削除
chara_num = data_4_8.shape[1]
sh = data_4_8.shape #行と列の数を取り出す
#shapeの列の数だけrangeで振り直してcolumnsに代入
data_4_8.columns = range(sh[1])
data_4_9 = np.split(data_4_8, com_num_4, 0)
np.shape(data_4_9)

(1141, 325, 85)

In [10]:
data_4_10 = data_4_9[0]
for num in range(1,int(com_num_4)):
    data_4_10 = np.concatenate([data_4_10, data_4_9[num]], 1)
data_4_11 = pd.DataFrame(data_4_10)
y = data_4_11.iloc[:,0:].to_numpy()
y.shape 

(325, 96985)

# 5. 形状変換後データの書き出し(pickleによる高速化)

In [11]:
returns_data_4.to_pickle('./inright_data-2.pickle') #ファイルの書き出し, 配当込み収益率はランク正規化をしていないデータであることに注意
data_4_11.to_pickle('./inleft_data-2.pickle') #ファイルの書き出し

In [12]:
print(os.path.getsize('./inright_data-2.pickle'), os.path.getsize('./inleft_data-2.pickle')) #ファイルの大きさを表示

2967171 252161577
